# Vector Maps in Python

## Vector map formats

reading shapefiles

In [ ]:
from fiona import collection
with collection('example_data/Kenya_admin_WGS84/Kenya_admin_2014_WGS84.shp', 'r') as input:
    for f in input:
        print(f)

{'properties': OrderedDict([('OBJECTID_1', 1), ('COUNTY', 'Turkana'), ('Province', 'Rift Valley'), ('ADM0', 'Kenya'), ('ADM3', None), ('ADM4', None), ('Shape_Leng', 15.1484195545), ('Shape_Area', 5.74380418891)]), 'id': '0', 'geometry': {'type': 'Polygon', 'coordinates': [[(35.328896592000035, 5.331331445000046), (35.334010000000035, 5.3299500000000535), (35.33929000000006, 5.330720000000042), (35.34168968400007, 5.331733200000031), (35.35324000000003, 5.336609999000075), (35.364052075000075, 5.34820378400002), (35.364720000000034, 5.348920000000021), (35.36625053900008, 5.349887747000025), (35.37432000100006, 5.354990000000043), (35.37656088800003, 5.355193558000053), (35.38268123200004, 5.355749519000028), (35.383336617000055, 5.35580905300003), (35.38761640400003, 5.356197821000023), (35.389713191000055, 5.356388289000051), (35.39703441700004, 5.357053337000025), (35.39997000000005, 5.3573200000000725), (35.40264522600006, 5.358423096000024), (35.407712665000076, 5.3605125920000205)

## converting map formats

shp_to_geojson: convert shapefile to geojson. Parameter is infile: name of shapefile (.shp).   Useful function from Martin Laloux, found on http://geospatialpython.com/2013/07/shapefile-to-geojson.html

In [ ]:
import ogr2ogr

ogr2ogr.main([‘’,'­f','GeoJSON','../test2.json','../TZwards.shp'])”

In [ ]:
def shp_to_geojson(infile):
    # read the shapefile
    reader = shapefile.Reader(infile)
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]
    buffer = []
    for sr in reader.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        buffer.append(dict(type="Feature", geometry=geom, properties=atr))

    # write the GeoJSON file
    outfile = infile[:-3]+"json"
    geojson = open(outfile, "w")
    geojson.write(json.dumps({"type": "FeatureCollection",
                             "features": buffer}, indent=2) + "\n")
    geojson.close()
    return()

## Location

Using a gazetteer to find a location for an address

Using point-in-polygon methods to find which (nonstandard) region a point is in.

### Getting lat/long from an address

Null Island is the point (0,0) just off the west coast of Africa.  It's the traditional place to put "no address" data. https://www.gislounge.com/null-island-where-geocoding-errors-go-to-live/ 

In [3]:
from geopy.geocoders import Nominatim
from geopy.geocoders import GoogleV3
geolocator = Nominatim()
googlocator = GoogleV3()

address = '1600 Pennsylvania Ave NW, Washington, DC'
result = geolocator.geocode(address, timeout=10)       
if result is None:            
    result = googlocator.geocode(address, timeout=10)        
if result is None:
    latlon = (0.0, 0.0)
else:
    latlon = (float(result.latitude), float(result.longitude))
print('Location: {}'.format(latlon))

Location: (38.8976989, -77.036553192281)


### Getting the distance between two points

Distance is a strange thing.  For example, there's as-the-crow-flies distance, but if you're travelling, you usually need (in a city) city-block distance, or a distance calculated from the distances along connecting roads. 

In [ ]:
from geopy.distance import vincenty
dist = vincenty(loc1, loc2)

# Getting distance from google maps and OSM
https://developers.google.com/maps/documentation/distance-matrix/intro#RequestParameters
http://py-googlemaps.sourceforge.net/

NB: you will need to "pip install googlemaps" from the terminal window before you can run this code.

In [2]:
from googlemaps import Client
gmaps = Client(api_key)
address = 'Constitution Ave NW & 10th St NW, Washington, DC'
destination = gmaps.latlng_to_address(38.887563, -77.019929)
local = gmaps.local_search('cafe near ' + destination)
directions = gmaps.directions(address, destination)

ImportError: cannot import name 'GoogleMaps'

In [3]:
import googlemaps
help(googlemaps)

Help on package googlemaps:

NAME
    googlemaps

DESCRIPTION
    # Copyright 2014 Google Inc. All rights reserved.
    #
    #
    # Licensed under the Apache License, Version 2.0 (the "License"); you may not
    # use this file except in compliance with the License. You may obtain a copy of
    # the License at
    #
    #     http://www.apache.org/licenses/LICENSE-2.0
    #
    # Unless required by applicable law or agreed to in writing, software
    # distributed under the License is distributed on an "AS IS" BASIS, WITHOUT
    # WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the
    # License for the specific language governing permissions and limitations under
    # the License.
    #

PACKAGE CONTENTS
    client
    convert
    directions
    distance_matrix
    elevation
    exceptions
    geocoding
    places
    roads
    timezone

CLASSES
    builtins.object
        googlemaps.client.Client
    
    class Client(builtins.object)
     |  Performs request